# BioExplorer - Metabolism - Combine movies into gallery view
![](../bioexplorer_metabolism_banner.png)

In [ ]:
from glob import glob
import os
from PIL import Image, ImageDraw, ImageFont

movies_folder = os.getenv('METABOLISM_MOVIE_FOLDER')

## Media production

### Create movies from frames

In [ ]:
all_files = glob(os.path.join(movies_folder, '*'))
all_folders = list()
for file in all_files:
    if os.path.isdir(file):
        all_folders.append(file)
for folder in all_folders:
    basename = os.path.basename(folder)
    movie_filename = os.path.join(movies_folder, basename + '_1k_v13.mp4')
    cmd = 'ffmpeg -f Image2 -i %s/%%*.png -b:v 16M -threads 16 -y %s 2>/dev/null' % (folder, movie_filename)
    print(basename)
    os.system(cmd)

### Combine movies into mosaic

In [ ]:
all_movie_files = glob(os.path.join(movies_folder, '*.mp4'))
movie_files=all_movie_files
print('Found %d movies' % len(movie_files))

In [ ]:
nb_columns = 5
nb_lines = 5
resolution=[3840, 2160]
image_size=[int(3840 / nb_columns), int(2160 / nb_lines)]

print(str(nb_columns) + 'x' + str(nb_lines) + '=' + str(nb_lines*nb_columns))
print(image_size)

all_movie_files = glob(os.path.join(movies_folder, '*.mp4'))

### Legend

In [ ]:
def add_text(draw, x_left, y_top, text, font_size, color):
    font = ImageFont.truetype('Roboto-Bold.ttf', size=font_size)
    draw.text((x_left,y_top), text, font=font, fill=color)

img_overlay = Image.new('RGBA', resolution, (0, 0, 0, 0)) 
draw_overlay = ImageDraw.Draw(img_overlay) 

i = 0
fontsize = 45
color = (255,255,255, 255)
for y in range(nb_lines):
    for x in range(nb_columns):
        x_left = x * image_size[0] + 5
        y_top = y * image_size[1] + 5
        text = os.path.basename(movie_files[i]).split('_')[0]
        i += 1

        add_text(draw_overlay, x_left, y_top, text, fontsize, color)

img_overlay.save(os.path.join(movies_folder, 'mosaic', 'metabolites_legend.png'))


### Background

In [ ]:
img_overlay = Image.new('RGBA', resolution, (0, 0, 0, 0)) 
draw_overlay = ImageDraw.Draw(img_overlay) 

img_background = Image.open(os.path.join(movies_folder, 'mosaic', 'metabolism_bioexplorer_2k_v13.png'))
img_background = img_background.resize(image_size)

for y in range(nb_lines):
    for x in range(nb_columns):
        x_left = x * image_size[0]
        y_top = y * image_size[1]
        img_overlay.paste(img_background, [x_left, y_top])

img_overlay.save(os.path.join(movies_folder, 'mosaic', 'metabolites_background.png'))


### Movie mosaic

In [ ]:

def generate_mosaic(files, output_filename):
    import subprocess
    nb_movies = len(files)

    values = list()
    values.append('ffmpeg')

    for movie_file in files:
        values.append('-i')
        values.append(movie_file)
    values.append('-filter_complex')

    filter_complex = 'nullsrc=size=' + str(resolution[0]) + 'x' + str(resolution[1]) + ' [base];'
    for i in range(nb_movies):
        filter_complex += ' [%d:v] setpts=PTS-STARTPTS, scale=%dx%d [vid%d];' % (i, image_size[0], image_size[1], i)

    for i in range(nb_movies):
        if i == 0:
            filter_complex += ' [base]'
        else:
            filter_complex += ' [tmp%d]' % (i - 1)
        filter_complex += '[vid%d] overlay=shortest=1:x=%d:y=%d' % (i, int(i % nb_columns) * image_size[0], (int(i / nb_columns) * image_size[1]))
        if i != nb_movies-1:
            filter_complex += ' [tmp%d];' % i

    values.append(filter_complex)
    values.append('-threads')
    values.append('20')
    values.append(output_filename)
    try:
        subprocess.run(
            values, 
            stdout=subprocess.DEVNULL,
            stderr=subprocess.STDOUT
        )
    except Exception as e:
        print(e)

concentrations_output_filename = os.path.join(movies_folder, 'mosaic', 'concentrations', '%05d.png')
background_output_filename = os.path.join(movies_folder, 'mosaic', 'background', '%05d.png')

generate_mosaic(all_movie_files, concentrations_output_filename)

### Extract frames from mosaic movie

In [ ]:
from PIL import Image
src_image = Image.open(os.path.join(movies_folder, 'mosaic', 'metabolism_bioexplorer_2k_v13.png'))

src_image.thumbnail(image_size)
dst_image = new_im = Image.new('RGBA', (3840,2160))

for i in range(nb_columns):
    for j in range(nb_lines):
        src_image=Image.eval(src_image,lambda x: x+(i+j)/30)
        dst_image.paste(src_image, (i * image_size[0],j * image_size[1]))

dst_image.save(os.path.join(movies_folder, 'mosaic', 'metabolism_gallery_4k_v13.png'))

In [ ]:
concentrations_output_frames = os.path.join(movies_folder, 'mosaic', 'concentrations', '%05d.png')
cmd = 'ffmpeg -i ' + concentrations_output_filename + ' ' + concentrations_output_frames + ' 2>/dev/null'
os.system(cmd)